# Import libraries

In [1]:
import numpy as np
import sys
import cv2
import pandas as pd 
from matplotlib import pyplot as plt
(CV2_VERSION, minor_ver, subminor_ver) = (cv2.__version__).split('.')
print('openCV version - ', CV2_VERSION)

openCV version -  4


# Define consts

In [2]:
CSV_NAME = 'list_regression.csv'
# out images will have dimensions of (width, height)
w = 300 
h = 300
DIM = (w, h)

# Generic functions

In [3]:
def get_frame_rate(video):
    # default frame rate - this values should be replace in the next if/else evaluation
    fps = 30 
    if int(CV2_VERSION) < 3:
        fps = video.get(cv2.cv.CV_CAP_PROP_FPS)
    else:
        fps = video.get(cv2.CAP_PROP_FPS)
    return round(fps, 2)


def between(start_ms, curr_s, end_ms):
    if end_ms == 'END':
        nend = 86400000.0 # BIGINT Number, represents 1 day... in python 3 there is no INT Max, in pyhton 2 it is 9223372036854775807, 
    else:
        nend = float(end_ms)
        
    start_s = start_ms / 1000
    end_s = nend / 1000
    
    return start_s <= curr_s <= end_s


def generate_out_videoname(vid):
    # check if the Video name is already formated
    if "video-" in vid:
        return vid.split('.')[0]
    
    out_video = 'video-00001' # default video name
    
    try:
        collision_number = vid.split('.')[0]
        numb = collision_number.split('collision')[1]
        s_numb = str(numb)
        while len(s_numb) < 5:
            s_numb = '0' + s_numb
        return f"video-{s_numb}"
    except:
        print(f"Exception generating new video name; returning name {out_video}")
        return out_video
    
    
def generate_framename(video_num,pos_frame):
    s_outvid = str(video_num)
    s_frame = str(pos_frame)
    
    while len(s_outvid) < 5:
        s_outvid = '0' + s_outvid
        
    while len(s_frame) < 5:
        s_frame = '0' + s_frame
    
    return f"video-{s_outvid}-frame-{s_frame}"


def generate_video_num(out_videoname):
    return int(out_videoname.split('-')[1])


# CSV functions

    Inside the list_regression.csv each row represents a video that you will want to write as a vector.
    Example of a line inside of list_regression.csv: 
        vid_name,ts0,tsf,x,y,z,u,v
    Where:
        vid_name: Name of video that contains collision
             ts0: Start time of the collision (in ms)
             tsf: End time of the collision (in ms)
               x: Vector to describe object's x axis. -1: right to left, 0: not moving, 1: left to right.
               y: Vector to describe object's y axis. -1: far to near, 0: not moving, 1: near to far.
               z: Vector to describe object's z axis. -1: up to down, 0: not moving, 1: down to up.
               u: Vertical area of image where collision occurs. -1 left side, 0 center, 1 right side.
               v: Horizontal area of image where collision occurs. -1 down side, 0 center, 1 up side.

In [10]:
df_video = pd.read_csv(CSV_NAME) 
df_video.head()

,vid_name,ts0,tsf,x,y,z,u,v
0,collision93.mp4,8000,END,1,-1,0,1,1
1,collision92.mp4,1000,END,1,-1,0,1,1
2,collision91.mp4,1000,END,1,-1,0,1,1
3,collision90.mp4,1000,END,1,-1,0,1,1
4,collision89.mp4,1000,END,1,-1,0,1,1


# Handle video

In [11]:
# For Debug purpose:
OUTOUT_INFO = False
OUTOUT_IMAGE = False

for index, df_row in df_video.iterrows():
    VIDEO_NAME = df_row['vid_name']
    
    video = cv2.VideoCapture(VIDEO_NAME)
    while not video.isOpened():
        video = cv2.VideoCapture(VIDEO_NAME)
        cv2.waitKey(1000)
        print("Wait for the header")
    
    out_videoname = generate_out_videoname(VIDEO_NAME)
    video_number = generate_video_num(out_videoname)
    
    # get first frame counter
    pos_frame = video.get(cv2.CAP_PROP_POS_FRAMES) 
    # get frames per second rate
    fps = get_frame_rate(video)
    
    df = pd.DataFrame(columns=['file', 'collision', 'x', 'y', 'z', 'u', 'v'])
    last_pos_frame = -1

    print(f'Processing video "{VIDEO_NAME}" that is {fps} fps')
    
    while True:
        flag, frame = video.read()    

        # The frame is ready and already captured
        if flag:
            # Get frame counter
            pos_frame = video.get(cv2.CAP_PROP_POS_FRAMES) 
            
            # Calculates the time elapsed in seconds
            time_elapsed = round((pos_frame / fps ), 2)     

            # Get current frame name
            curr_frame_name = generate_framename(video_number, int(pos_frame))
            
            # Check if there was collision
            is_collision = between(df_row['ts0'], time_elapsed, df_row['tsf'])

            # Populate dataframe row
            if is_collision:
                new_row = {
                    'file' : curr_frame_name,
                    'collision': 1, 
                    'x': df_row['x'],
                    'y': df_row['y'],
                    'z': df_row['z'],
                    'u': df_row['u'],
                    'v': df_row['v']
                }
            else:
                new_row = {
                    'file' : curr_frame_name,
                    'collision': 0, 
                    'x': 0,
                    'y': 0,
                    'z': 0,
                    'u': 0,
                    'v': 0
                }
                
            # Add row to dataframe
            df = df.append(new_row, ignore_index=True)
            
            # Print info header on dataframe
            print(f'Frame {int(pos_frame)} at time {time_elapsed}s -> {("","NO ")[is_collision]}Collision')
            
            # Save image to file
            resized_frame = cv2.resize(frame, DIM, interpolation = cv2.INTER_AREA)
            cv2.imwrite(f"./images/{curr_frame_name}.png", resized_frame) 
            
            # Plot info frame
            if OUTOUT_INFO:
                print(new_row)
            
            # Plot current frame
            if OUTOUT_IMAGE:
                plt.imshow(resized_frame)
                plt.pause(0.05)
                plt.show()

            # Set last_post_frame, this allows to recognize end of video
            last_pos_frame = pos_frame

        else:
            # Check that we have reached end of video
            if int(pos_frame) == int(last_pos_frame):
                print("\n-----------------------------------------------------------------\n")
                break

        if cv2.waitKey(10) == 27:
            break
        
    # dropping duplicate values 
    # df.drop_duplicates(keep='first',inplace=True)
    df.to_excel(f'dataframes/{out_videoname}.xlsx',index=False)      

Processing video "collision93.mp4" that is 29.97 fps
Frame 1 at time 0.03s -> Collision
Frame 2 at time 0.07s -> Collision
Frame 3 at time 0.1s -> Collision
Frame 4 at time 0.13s -> Collision
Frame 5 at time 0.17s -> Collision
Frame 6 at time 0.2s -> Collision
Frame 7 at time 0.23s -> Collision
Frame 8 at time 0.27s -> Collision
Frame 9 at time 0.3s -> Collision
Frame 10 at time 0.33s -> Collision
Frame 11 at time 0.37s -> Collision
Frame 12 at time 0.4s -> Collision
Frame 13 at time 0.43s -> Collision
Frame 14 at time 0.47s -> Collision
Frame 15 at time 0.5s -> Collision
Frame 16 at time 0.53s -> Collision
Frame 17 at time 0.57s -> Collision
Frame 18 at time 0.6s -> Collision
Frame 19 at time 0.63s -> Collision
Frame 20 at time 0.67s -> Collision
Frame 21 at time 0.7s -> Collision
Frame 22 at time 0.73s -> Collision
Frame 23 at time 0.77s -> Collision
Frame 24 at time 0.8s -> Collision
Frame 25 at time 0.83s -> Collision
Frame 26 at time 0.87s -> Collision
Frame 27 at time 0.9s -> Col

Frame 231 at time 7.71s -> Collision
Frame 232 at time 7.74s -> Collision
Frame 233 at time 7.77s -> Collision
Frame 234 at time 7.81s -> Collision
Frame 235 at time 7.84s -> Collision
Frame 236 at time 7.87s -> Collision
Frame 237 at time 7.91s -> Collision
Frame 238 at time 7.94s -> Collision
Frame 239 at time 7.97s -> Collision
Frame 240 at time 8.01s -> NO Collision
Frame 241 at time 8.04s -> NO Collision
Frame 242 at time 8.07s -> NO Collision
Frame 243 at time 8.11s -> NO Collision
Frame 244 at time 8.14s -> NO Collision
Frame 245 at time 8.17s -> NO Collision
Frame 246 at time 8.21s -> NO Collision
Frame 247 at time 8.24s -> NO Collision
Frame 248 at time 8.27s -> NO Collision
Frame 249 at time 8.31s -> NO Collision
Frame 250 at time 8.34s -> NO Collision
Frame 251 at time 8.38s -> NO Collision
Frame 252 at time 8.41s -> NO Collision
Frame 253 at time 8.44s -> NO Collision
Frame 254 at time 8.48s -> NO Collision
Frame 255 at time 8.51s -> NO Collision
Frame 256 at time 8.54s -> 

Frame 128 at time 4.27s -> NO Collision
Frame 129 at time 4.31s -> NO Collision
Frame 130 at time 4.34s -> NO Collision
Frame 131 at time 4.37s -> NO Collision
Frame 132 at time 4.41s -> NO Collision
Frame 133 at time 4.44s -> NO Collision
Frame 134 at time 4.47s -> NO Collision
Frame 135 at time 4.51s -> NO Collision
Frame 136 at time 4.54s -> NO Collision

-----------------------------------------------------------------

Processing video "collision90.mp4" that is 29.95 fps
Frame 1 at time 0.03s -> Collision
Frame 2 at time 0.07s -> Collision
Frame 3 at time 0.1s -> Collision
Frame 4 at time 0.13s -> Collision
Frame 5 at time 0.17s -> Collision
Frame 6 at time 0.2s -> Collision
Frame 7 at time 0.23s -> Collision
Frame 8 at time 0.27s -> Collision
Frame 9 at time 0.3s -> Collision
Frame 10 at time 0.33s -> Collision
Frame 11 at time 0.37s -> Collision
Frame 12 at time 0.4s -> Collision
Frame 13 at time 0.43s -> Collision
Frame 14 at time 0.47s -> Collision
Frame 15 at time 0.5s -> Col

Frame 202 at time 6.74s -> NO Collision
Frame 203 at time 6.78s -> NO Collision
Frame 204 at time 6.81s -> NO Collision
Frame 205 at time 6.84s -> NO Collision
Frame 206 at time 6.88s -> NO Collision
Frame 207 at time 6.91s -> NO Collision
Frame 208 at time 6.94s -> NO Collision
Frame 209 at time 6.98s -> NO Collision
Frame 210 at time 7.01s -> NO Collision
Frame 211 at time 7.05s -> NO Collision
Frame 212 at time 7.08s -> NO Collision
Frame 213 at time 7.11s -> NO Collision
Frame 214 at time 7.15s -> NO Collision
Frame 215 at time 7.18s -> NO Collision
Frame 216 at time 7.21s -> NO Collision
Frame 217 at time 7.25s -> NO Collision
Frame 218 at time 7.28s -> NO Collision
Frame 219 at time 7.31s -> NO Collision
Frame 220 at time 7.35s -> NO Collision
Frame 221 at time 7.38s -> NO Collision
Frame 222 at time 7.41s -> NO Collision
Frame 223 at time 7.45s -> NO Collision
Frame 224 at time 7.48s -> NO Collision
Frame 225 at time 7.51s -> NO Collision
Frame 226 at time 7.55s -> NO Collision


Frame 176 at time 5.88s -> NO Collision
Frame 177 at time 5.91s -> NO Collision
Frame 178 at time 5.95s -> NO Collision

-----------------------------------------------------------------

